In [1]:
import sqlite3

# Path to your SQLite database file.
db_path = "FoodDataCentral.db"

# Data Cleaning

In [3]:
import os
import pandas as pd

# Folder with the original CSV files.
csv_folder = "FoodData_Central_csv_2024-10-31"

# Folder to store cleaned CSV files.
clean_folder = "FoodData_Central_csv_cleaned"
os.makedirs(clean_folder, exist_ok=True)

# Function to clean a DataFrame for common issues.
def clean_dataframe(df):
    # Example: If the column "FDC ID" exists, remove rows where it is null or empty.
    if "FDC ID" in df.columns:
        initial_count = len(df)
        df = df[df["FDC ID"].notnull() & (df["FDC ID"].astype(str).str.strip() != "")]
        removed = initial_count - len(df)
        print(f"  Removed {removed} rows with missing FDC ID.")
    
    # Add other cleaning rules here as needed.
    # For example, you might want to trim spaces from all string columns:
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].str.strip()
    
    return df

# Loop over all CSV files in the source folder.
for filename in os.listdir(csv_folder):
    if filename.lower().endswith(".csv"):
        file_path = os.path.join(csv_folder, filename)
        print(f"Processing file: {file_path}")
        try:
            # Read the CSV file. Adjust low_memory if needed.
            df = pd.read_csv(file_path, low_memory=False, on_bad_lines='skip')
            df = clean_dataframe(df)
            
            # Save the cleaned DataFrame to the new folder with the same filename.
            cleaned_file_path = os.path.join(clean_folder, filename)
            df.to_csv(cleaned_file_path, index=False)
            print(f"Cleaned file saved to: {cleaned_file_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

print("CSV cleaning complete.")


Processing file: FoodData_Central_csv_2024-10-31\acquisition_samples.csv
Cleaned file saved to: FoodData_Central_csv_cleaned\acquisition_samples.csv
Processing file: FoodData_Central_csv_2024-10-31\agricultural_samples.csv
Cleaned file saved to: FoodData_Central_csv_cleaned\agricultural_samples.csv
Processing file: FoodData_Central_csv_2024-10-31\branded_food.csv
Cleaned file saved to: FoodData_Central_csv_cleaned\branded_food.csv
Processing file: FoodData_Central_csv_2024-10-31\fndds_derivation.csv
Cleaned file saved to: FoodData_Central_csv_cleaned\fndds_derivation.csv
Processing file: FoodData_Central_csv_2024-10-31\fndds_ingredient_nutrient_value.csv
  Removed 4989 rows with missing FDC ID.


C:\Users\ButterBoy\AppData\Local\Temp\ipykernel_4836\3934834198.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.strip()


Cleaned file saved to: FoodData_Central_csv_cleaned\fndds_ingredient_nutrient_value.csv
Processing file: FoodData_Central_csv_2024-10-31\food.csv
Cleaned file saved to: FoodData_Central_csv_cleaned\food.csv
Processing file: FoodData_Central_csv_2024-10-31\food_attribute.csv
Cleaned file saved to: FoodData_Central_csv_cleaned\food_attribute.csv
Processing file: FoodData_Central_csv_2024-10-31\food_attribute_type.csv
Cleaned file saved to: FoodData_Central_csv_cleaned\food_attribute_type.csv
Processing file: FoodData_Central_csv_2024-10-31\food_calorie_conversion_factor.csv
Cleaned file saved to: FoodData_Central_csv_cleaned\food_calorie_conversion_factor.csv
Processing file: FoodData_Central_csv_2024-10-31\food_category.csv
Cleaned file saved to: FoodData_Central_csv_cleaned\food_category.csv
Processing file: FoodData_Central_csv_2024-10-31\food_component.csv
Cleaned file saved to: FoodData_Central_csv_cleaned\food_component.csv
Processing file: FoodData_Central_csv_2024-10-31\food_nutr

# Clean fndds bad FDC IDs

In [4]:
import os
import pandas as pd

# Folder containing the cleaned CSV files.
csv_folder = "FoodData_Central_csv_cleaned"

# Read the food.csv file.
food_csv = os.path.join(csv_folder, "food.csv")
df_food = pd.read_csv(food_csv, low_memory=False)
df_food["fdc_id"] = df_food["fdc_id"].astype(str).str.strip()

# Create a set of valid FDC IDs.
valid_ids = set(df_food["fdc_id"].dropna())
print(f"Found {len(valid_ids)} valid FDC IDs in food.csv")
print("Sample valid IDs:", list(valid_ids)[:5])

# Read the fndds_ingredient_nutrient_value CSV.
fnd_csv = os.path.join(csv_folder, "fndds_ingredient_nutrient_value.csv")
df_fnd = pd.read_csv(fnd_csv, low_memory=False)

# Ensure the CSV has the "FDC ID" column.
if "FDC ID" not in df_fnd.columns:
    raise ValueError("Column 'FDC ID' not found in fndds_ingredient_nutrient_value.csv")

# Define a function to normalize FDC ID values.
def normalize_fdc(value):
    try:
        # Convert the value to float then to int, and back to string.
        # This will remove any trailing .0.
        return str(int(float(value)))
    except Exception as e:
        return str(value).strip()

# Normalize the "FDC ID" column in df_fnd.
df_fnd["FDC ID"] = df_fnd["FDC ID"].apply(normalize_fdc)
print("Sample normalized FDC IDs in fndds_ingredient_nutrient_value:", df_fnd["FDC ID"].unique()[:5])

# Filter rows: keep only those where the normalized "FDC ID" exists in the valid_ids.
df_filtered = df_fnd[df_fnd["FDC ID"].isin(valid_ids)]
dropped_count = len(df_fnd) - len(df_filtered)
print(f"Dropped {dropped_count} rows from fndds_ingredient_nutrient_value.csv due to missing FDC IDs in food.csv.")

# Save the filtered DataFrame to a new CSV file.
output_csv = os.path.join(csv_folder, "fndds_ingredient_nutrient_value.csv")
df_filtered.to_csv(output_csv, index=False)
print(f"Filtered CSV saved to {output_csv}")


Found 2063876 valid FDC IDs in food.csv
Sample valid IDs: ['1660367', '1545897', '1234295', '172679', '569879']
Sample normalized FDC IDs in fndds_ingredient_nutrient_value: ['173410' '173411' '173412' '172175' '172176']
Dropped 7218 rows from fndds_ingredient_nutrient_value.csv due to missing FDC IDs in food.csv.
Filtered CSV saved to FoodData_Central_csv_cleaned\fndds_ingredient_nutrient_value.csv


# Clean Bad Food Calorie Conversion Foreign Key

In [6]:
import os
import pandas as pd

# Define a function to normalize a value (e.g., "22721.0" becomes "22721").
def normalize_value(value):
    try:
        return str(int(float(value)))
    except Exception as e:
        return str(value).strip()

# Folder containing the cleaned CSV files.
csv_folder = "FoodData_Central_csv_cleaned"

# ------------------------------
# Step 1: Read the parent CSV file.
# ------------------------------
parent_csv = os.path.join(csv_folder, "food_nutrient_conversion_factor.csv")
df_parent = pd.read_csv(parent_csv, low_memory=False)
df_parent["id"] = df_parent["id"].astype(str).apply(normalize_value)
valid_ids = set(df_parent["id"].dropna())
print(f"Found {len(valid_ids)} valid IDs in food_nutrient_conversion_factor.csv")

# ------------------------------
# Step 2: Read the child CSV file.
# ------------------------------
child_csv = os.path.join(csv_folder, "food_calorie_conversion_factor.csv")
df_child = pd.read_csv(child_csv, low_memory=False)
if "food_nutrient_conversion_factor_id" not in df_child.columns:
    raise ValueError("Column 'food_nutrient_conversion_factor_id' not found in food_calorie_conversion_factor CSV.")

# Normalize the foreign key values.
df_child["food_nutrient_conversion_factor_id"] = df_child["food_nutrient_conversion_factor_id"].astype(str).apply(normalize_value)

# ------------------------------
# Step 3: Filter the child DataFrame.
# ------------------------------
initial_count = len(df_child)
df_filtered = df_child[df_child["food_nutrient_conversion_factor_id"].isin(valid_ids)]
filtered_count = len(df_filtered)
print(f"Dropped {initial_count - filtered_count} rows from food_calorie_conversion_factor.csv due to invalid foreign keys.")

# ------------------------------
# Step 4: Save the filtered DataFrame to a new CSV file.
# ------------------------------
output_csv = os.path.join(csv_folder, "food_calorie_conversion_factor.csv")
df_filtered.to_csv(output_csv, index=False)
print(f"Filtered CSV saved to {output_csv}")


Found 658 valid IDs in food_nutrient_conversion_factor.csv
Dropped 4711 rows from food_calorie_conversion_factor.csv due to invalid foreign keys.
Filtered CSV saved to FoodData_Central_csv_cleaned\food_calorie_conversion_factor.csv


# Clean bad foreign keys, sub samples & wweia

In [7]:
import os
import pandas as pd

# Define a function to normalize FDC values.
def normalize_fdc(value):
    try:
        # Convert to float then to int, then back to string.
        return str(int(float(value)))
    except Exception as e:
        return str(value).strip()

# Folder containing the cleaned CSV files.
csv_folder = "FoodData_Central_csv_cleaned"

#############################
# 1. Filter sub_sample_food #
#############################

# Read food.csv to get valid food IDs.
food_csv = os.path.join(csv_folder, "food.csv")
df_food = pd.read_csv(food_csv, low_memory=False)
df_food["fdc_id"] = df_food["fdc_id"].astype(str).apply(normalize_fdc)
valid_food_ids = set(df_food["fdc_id"].dropna())
print(f"[sub_sample_food] Found {len(valid_food_ids)} valid FDC IDs in food.csv.")

# Read sub_sample_food.csv.
sub_sample_food_csv = os.path.join(csv_folder, "sub_sample_food.csv")
df_sub = pd.read_csv(sub_sample_food_csv, low_memory=False)
# Normalize key columns.
df_sub["fdc_id"] = df_sub["fdc_id"].astype(str).apply(normalize_fdc)
df_sub["fdc_id_of_sample_food"] = df_sub["fdc_id_of_sample_food"].astype(str).apply(normalize_fdc)

# Filter: keep rows where both keys are in valid_food_ids.
mask_sub = df_sub["fdc_id"].isin(valid_food_ids) & df_sub["fdc_id_of_sample_food"].isin(valid_food_ids)
df_sub_filtered = df_sub[mask_sub]
dropped_sub = len(df_sub) - len(df_sub_filtered)
print(f"[sub_sample_food] Dropped {dropped_sub} rows due to invalid FDC IDs.")
output_csv_sub = os.path.join(csv_folder, "sub_sample_food.csv")
df_sub_filtered.to_csv(output_csv_sub, index=False)
print(f"[sub_sample_food] Filtered CSV saved to {output_csv_sub}")

#################################
# 2. Filter sub_sample_result   #
#################################

# Read food_nutrient.csv to get valid food_nutrient IDs.
fn_csv = os.path.join(csv_folder, "food_nutrient.csv")
df_fn = pd.read_csv(fn_csv, low_memory=False)
df_fn["id"] = df_fn["id"].astype(str).apply(normalize_fdc)
valid_fn_ids = set(df_fn["id"].dropna())
print(f"[sub_sample_result] Found {len(valid_fn_ids)} valid food_nutrient IDs.")

# Read lab_method.csv to get valid lab_method IDs.
lm_csv = os.path.join(csv_folder, "lab_method.csv")
df_lm = pd.read_csv(lm_csv, low_memory=False)
df_lm["id"] = df_lm["id"].astype(str).apply(normalize_fdc)
valid_lm_ids = set(df_lm["id"].dropna())
print(f"[sub_sample_result] Found {len(valid_lm_ids)} valid lab_method IDs.")

# Read sub_sample_result.csv.
sub_sample_result_csv = os.path.join(csv_folder, "sub_sample_result.csv")
df_sub_result = pd.read_csv(sub_sample_result_csv, low_memory=False)
df_sub_result["food_nutrient_id"] = df_sub_result["food_nutrient_id"].astype(str).apply(normalize_fdc)
df_sub_result["lab_method_id"] = df_sub_result["lab_method_id"].astype(str).apply(normalize_fdc)

mask_result = df_sub_result["food_nutrient_id"].isin(valid_fn_ids) & df_sub_result["lab_method_id"].isin(valid_lm_ids)
df_sub_result_filtered = df_sub_result[mask_result]
dropped_result = len(df_sub_result) - len(df_sub_result_filtered)
print(f"[sub_sample_result] Dropped {dropped_result} rows due to invalid food_nutrient_id or lab_method_id.")
output_csv_result = os.path.join(csv_folder, "sub_sample_result.csv")
df_sub_result_filtered.to_csv(output_csv_result, index=False)
print(f"[sub_sample_result] Filtered CSV saved to {output_csv_result}")

#################################
# 3. Filter survey_fndds_food   #
#################################

# Read survey_fndds_food.csv.
survey_csv = os.path.join(csv_folder, "survey_fndds_food.csv")
df_survey = pd.read_csv(survey_csv, low_memory=False)
df_survey["fdc_id"] = df_survey["fdc_id"].astype(str).apply(normalize_fdc)

# Filter survey data by valid food IDs.
mask_survey = df_survey["fdc_id"].isin(valid_food_ids)
df_survey_filtered = df_survey[mask_survey]
print(f"[survey_fndds_food] After fdc_id filter: {len(df_survey_filtered)} rows remain out of {len(df_survey)}.")

# Read wweia_food_category.csv to get valid category codes.
wweia_csv = os.path.join(csv_folder, "wweia_food_category.csv")
df_wweia = pd.read_csv(wweia_csv, low_memory=False)
# Assume the primary key column is "wweia_food_category" (or "wweia_food_category_code")
if "wweia_food_category" in df_wweia.columns:
    df_wweia["wweia_food_category"] = df_wweia["wweia_food_category"].astype(str).str.strip()
    valid_wweia = set(df_wweia["wweia_food_category"].dropna())
elif "wweia_food_category_code" in df_wweia.columns:
    df_wweia["wweia_food_category_code"] = df_wweia["wweia_food_category_code"].astype(str).str.strip()
    valid_wweia = set(df_wweia["wweia_food_category_code"].dropna())
else:
    valid_wweia = set()
    print("[survey_fndds_food] Warning: Could not determine valid WWEIA category codes.")

# Normalize and filter the survey file by the WWEIA category.
if "wweia_category_code" in df_survey_filtered.columns:
    df_survey_filtered["wweia_category_code"] = df_survey_filtered["wweia_category_code"].astype(str).str.strip()
    mask_wweia = df_survey_filtered["wweia_category_code"].isin(valid_wweia)
    df_survey_filtered = df_survey_filtered[mask_wweia]
    print(f"[survey_fndds_food] After WWEIA category filtering: {len(df_survey_filtered)} rows remain.")
else:
    print("[survey_fndds_food] Column 'wweia_category_code' not found in survey_fndds_food CSV.")

output_csv_survey = os.path.join(csv_folder, "survey_fndds_food.csv")
df_survey_filtered.to_csv(output_csv_survey, index=False)
print(f"[survey_fndds_food] Filtered CSV saved to {output_csv_survey}")

print("CSV filtering complete.")


[sub_sample_food] Found 2063876 valid FDC IDs in food.csv.
[sub_sample_food] Dropped 2 rows due to invalid FDC IDs.
[sub_sample_food] Filtered CSV saved to FoodData_Central_csv_cleaned\sub_sample_food.csv
[sub_sample_result] Found 26848323 valid food_nutrient IDs.
[sub_sample_result] Found 283 valid lab_method IDs.
[sub_sample_result] Dropped 17 rows due to invalid food_nutrient_id or lab_method_id.
[sub_sample_result] Filtered CSV saved to FoodData_Central_csv_cleaned\sub_sample_result.csv
[survey_fndds_food] After fdc_id filter: 5432 rows remain out of 5432.
[survey_fndds_food] After WWEIA category filtering: 5432 rows remain.
[survey_fndds_food] Filtered CSV saved to FoodData_Central_csv_cleaned\survey_fndds_food.csv
CSV filtering complete.


# Make the original Database as from USDA food

In [10]:
# Connect to (or create) the database and enable foreign key support.
conn = sqlite3.connect(db_path)
conn.execute("PRAGMA foreign_keys = ON;")
cur = conn.cursor()


cur.execute("""
CREATE TABLE IF NOT EXISTS food (
    fdc_id INTEGER PRIMARY KEY,         -- Unique permanent identifier of the food (FDC Source ID)
    foodClass TEXT,                     -- For internal use only
    data_type TEXT,                     -- Type of food data (see Files tab for possible values)
    description TEXT,                   -- Description of the food
    food_category_id TEXT,              -- ID of the food category the food belongs to
    publication_date TEXT,              -- Date when the food was published to FoodData Central
    scientific_name TEXT                -- The scientific name for the food
);
""")
conn.commit()
print("food table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS measure_unit (
    id INTEGER PRIMARY KEY,  -- Unique permanent identifier for the measure unit
    name TEXT               -- Name of the unit (e.g., "cup", "tsp", "g", etc.)
);
""")
conn.commit()
print("measure_unit table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS nutrient (
    id INTEGER PRIMARY KEY,             -- Unique permanent identifier for the nutrient
    name TEXT,                          -- Name of the nutrient
    unit_name TEXT,                     -- Standard unit of measure for the nutrient (per 100g of food)
    nutrient_nbr TEXT,                  -- A unique code identifying the nutrient or food constituent
    rank INTEGER                        -- Alternate ordering value for nutrients
);
""")
conn.commit()
print("nutrient table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS agricultural_samples (
    fdc_id INTEGER PRIMARY KEY,      -- ID of the food in the FoodData table (global key)
    acquisition_date TEXT,           -- The date this food was obtained
    market_class TEXT,               -- The specific kind of this food (e.g., "Pinto")
    treatment TEXT,                  -- Any special production condition (e.g., "drought" or "control")
    state TEXT,                      -- The state in which this food was produced
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
);
""")

conn.commit()
print("agricultural_samples table created.")



cur.execute("""
CREATE TABLE IF NOT EXISTS acquisition_samples (
    id INTEGER PRIMARY KEY,                              -- Unique permanent identifier for this record
    fdc_id_of_sample_food INTEGER,                       -- ID of the sample food that uses the acquisitioned food
    fdc_id_of_acquisition_food INTEGER,                  -- ID of the acquisitioned food used in the sample food
    FOREIGN KEY (fdc_id_of_sample_food) REFERENCES food(fdc_id),
    FOREIGN KEY (fdc_id_of_acquisition_food) REFERENCES food(fdc_id)
);
""")

conn.commit()
print("acquisition_samples table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS branded_food (
    fdc_id INTEGER PRIMARY KEY,                -- ID of the food in the food table (FDC Source ID)
    brand_owner TEXT,                          -- Brand owner for the food
    brand_name TEXT,                           -- Brand name for the food
    subbrand_name TEXT,                        -- Subbrand and variation descriptions
    gtin_upc TEXT,                             -- GTIN or UPC code identifying the food; duplicates indicate updates
    ingredients TEXT,                          -- The list of ingredients as it appears on the product label
    not_a_significant_source_of TEXT,          -- Full text for the "not a significant source of…" label claim
    serving_size REAL,                         -- The amount of the serving size when expressed as gram or ml
    serving_size_unit TEXT,                    -- The unit used to express the serving size (gram or ml)
    household_serving_fulltext TEXT,           -- The amount and unit of serving size when expressed in household units
    branded_food_category TEXT,                -- The category of the branded food (assigned by GDSN or Label Insight)
    data_source TEXT,                          -- The source of the data (GDSN for GS1 or LI for Label Insight)
    package_weight REAL,                       -- Weight of the package
    modified_date TEXT,                        -- Date when the product data was last modified by the data provider
    available_date TEXT,                       -- Date when the product record was available for inclusion in the database
    discontinued_date TEXT,                    -- Date when the product was discontinued
    market_country TEXT,                       -- The primary country where the product is marketed
    preparation_state_code TEXT,               -- Code describing the preparation of the food (as defined in GS1)
    trade_channel TEXT,                        -- List of locations or programs in which the food is available
    short_description TEXT,                    -- Manufacturer's short description of the product
    material_code TEXT,                         -- The material code for the food, if one is present
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
);
""")

conn.commit()
print("branded_food table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS fndds_derivation (
    id INTEGER PRIMARY KEY,                 -- Unique permanent identifier for this derivation record
    "derivation code" TEXT,                   -- Derivation code as defined by FDC
    "derivation description" TEXT             -- Description of the derivation code
);
""")

conn.commit()
print("fndds_derivation table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS fndds_ingredient_nutrient_value (
    id INTEGER PRIMARY KEY AUTOINCREMENT,     -- Unique permanent identifier for this record
    "ingredient code" TEXT,                   -- Identifies only NDB number
    "Ingredient description" TEXT,            -- Description of NDB number
    "Nutrient code" TEXT,                     -- 3-digit identification number
    "Nutrient value" REAL,                    -- Amount per 100g edible portion for energy and 64 nutrients
    "Nutrient value source" TEXT,             -- FDC or other source for nutrient value
    "FDC ID" INTEGER,                         -- Identifier of food in FDC
    "Derivation code" TEXT,                   -- Derivation code as defined by FDC
    "SR AddMod year" TEXT,                    -- Year value added or last modified as defined by SR
    "Foundation year acquired" TEXT,          -- Initial year acquired as defined by FDC
    "Start date" TEXT,                        -- Start date of FNDDS version released
    "End date" TEXT,                          -- End date of FNDDS version released
    FOREIGN KEY ("FDC ID") REFERENCES food(fdc_id)
);
""")
conn.commit()
print("fndds_ingredient_nutrient_value table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS food_attribute (
    id INTEGER PRIMARY KEY,                           -- Unique permanent identifier for the food attribute
    fdc_id INTEGER NOT NULL,                          -- ID of the food this attribute pertains to
    seq_num INTEGER,                                  -- Display order of the attribute
    food_attribute_type_id INTEGER,                   -- ID of the type of food attribute
    name TEXT,                                        -- Name of the food attribute
    value TEXT,                                       -- The actual value of the attribute
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
);
""")
conn.commit()
print("food_attribute table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS food_attribute_type (
    id INTEGER PRIMARY KEY,           -- Unique permanent identifier for the attribute type
    name TEXT,                        -- Name of the attribute associated with the food; should be displayable to users
    description TEXT                  -- Description of the attribute
);
""")
conn.commit()
print("food_attribute_type table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS food_nutrient_conversion_factor (
    id INTEGER PRIMARY KEY,  -- Unique permanent identifier for this conversion factor record
    fdc_id INTEGER,          -- ID of the food that this conversion factor pertains to
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
);
""")
conn.commit()
print("food_nutrient_conversion_factor table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS food_calorie_conversion_factor (
    food_nutrient_conversion_factor_id INTEGER PRIMARY KEY,  -- Unique identifier for the conversion factor record
    protein_value REAL,                                      -- Multiplication factor for protein
    fat_value REAL,                                          -- Multiplication factor for fat
    carbohydrate_value REAL,                                 -- Multiplication factor for carbohydrates
    FOREIGN KEY (food_nutrient_conversion_factor_id) REFERENCES food_nutrient_conversion_factor(id)
);
""")
conn.commit()
print("food_calorie_conversion_factor table created.")



cur.execute("""
CREATE TABLE IF NOT EXISTS food_category (
    id INTEGER PRIMARY KEY,        -- Unique permanent identifier for the food category
    code TEXT,                     -- Food group code
    description TEXT               -- Description of the food group
);
""")
conn.commit()
print("food_category table created.")



cur.execute("""
CREATE TABLE IF NOT EXISTS food_component (
    id INTEGER PRIMARY KEY,           -- Unique permanent identifier for a food component record
    fdc_id INTEGER NOT NULL,          -- ID of the food this component pertains to
    name TEXT,                        -- The kind of component, e.g. bone
    pct_weight REAL,                  -- The weight of the component as a percentage of the total food weight
    is_refuse TEXT,                   -- Whether the component is considered refuse (non-edible)
    gram_weight REAL,                 -- The weight of the component in grams
    data_points INTEGER,              -- Number of observations on which this measure is based
    min_year_acquired INTEGER,        -- Minimum purchase year of acquisitions used to derive the component value
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
);
""")
conn.commit()
print("food_component table created.")




cur.execute("""
CREATE TABLE IF NOT EXISTS food_nutrient (
    id INTEGER PRIMARY KEY,                     -- Unique permanent identifier for the nutrient record
    fdc_id INTEGER NOT NULL,                    -- ID of the food this nutrient pertains to
    nutrient_id INTEGER,                        -- ID of the nutrient (references nutrient(id))
    amount REAL,                                -- Amount of the nutrient per 100g of food
    data_points INTEGER,                        -- Number of observations on which the value is based
    derivation_id INTEGER,                      -- ID of the derivation technique used to derive the nutrient value
    min REAL,                                   -- Minimum amount
    max REAL,                                   -- Maximum amount
    median REAL,                                -- Median amount
    loq REAL,                                   -- Limit of quantification provided by laboratory
    footnote TEXT,                              -- Comments on any unusual aspects of the nutrient value
    min_year_acquired INTEGER,                  -- Minimum purchase year of all acquisitions used to derive the nutrient value
    percent_daily_value REAL,                   -- Percentage of the recommended daily value
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id),
    FOREIGN KEY (nutrient_id) REFERENCES nutrient(id)
);
""")
conn.commit()
print("food_nutrient table created.")





cur.execute("""
CREATE TABLE IF NOT EXISTS food_nutrient_derivation (
    id INTEGER PRIMARY KEY,              -- Unique permanent identifier for the derivation record
    code TEXT,                           -- Code used for the derivation (e.g. "A" means analytical)
    description TEXT                     -- Description of the derivation
);
""")
conn.commit()
print("food_nutrient_derivation table created.")



cur.execute("""
CREATE TABLE IF NOT EXISTS food_nutrient_source (
    id INTEGER PRIMARY KEY,        -- Unique permanent identifier for the nutrient source
    code TEXT,                     -- Code used for the source (e.g. "4" for calculated or imputed)
    description TEXT               -- Description of the nutrient source
);
""")
conn.commit()
print("food_nutrient_source table created.")



cur.execute("""
CREATE TABLE IF NOT EXISTS food_portion (
    id INTEGER PRIMARY KEY,          -- Unique permanent identifier for this food portion record
    fdc_id INTEGER NOT NULL,         -- ID of the food this portion pertains to
    seq_num INTEGER,                 -- The order in which the measure will be displayed
    amount REAL,                     -- The number of measure units that comprise the measure
    measure_unit_id INTEGER,            -- The unit used for the measure (e.g., tsp, cup) as defined in the CSV
    portion_description TEXT,        -- Description of the portion (e.g., "1 slice", "1 cup")
    modifier TEXT,                   -- Qualifier for the measure (e.g., "diced", "crushed")
    gram_weight REAL,                -- Weight of the measure in grams
    data_points INTEGER,             -- Number of observations on which the measure is based
    footnote TEXT,                   -- Comments on any unusual aspects of the measure
    min_year_acquired INTEGER,       -- Minimum purchase year of acquisitions used to derive the measure value
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id),
    FOREIGN KEY (measure_unit_id) REFERENCES measure_unit(id)
);
""")
conn.commit()
print("food_portion table created.")



cur.execute("""
CREATE TABLE IF NOT EXISTS food_protein_conversion_factor (
    id INTEGER PRIMARY KEY AUTOINCREMENT,    -- Unique permanent identifier for this record
    food_nutrient_conversion_factor_id INTEGER,  -- References the related row in nutrient_conversion_factor
    value REAL,                                  -- Multiplication factor used to calculate protein from nitrogen
    FOREIGN KEY (food_nutrient_conversion_factor_id) REFERENCES food_nutrient_conversion_factor(id)
);
""")
conn.commit()
print("food_protein_conversion_factor table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS food_update_log_entry (
    id INTEGER PRIMARY KEY AUTOINCREMENT,  -- Unique permanent identifier for this update log record
    fdc_id INTEGER,                -- ID of the food in the food table
    description TEXT,                       -- Description of the food
    last_updated TEXT,              -- Date when the food was published to FoodData Central
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
);
""")
conn.commit()
print("food_update_log_entry table created.")



cur.execute("""
CREATE TABLE IF NOT EXISTS sr_legacy_food (
    fdc_id INTEGER PRIMARY KEY,         -- Unique permanent identifier of the food in the food table
    NDB_number INTEGER UNIQUE,                    -- Unique number assigned for final food, starting from the minimum number of 100,000
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
);
""")
conn.commit()
print("sr_legacy_food table created.")




cur.execute("""
CREATE TABLE IF NOT EXISTS foundation_food (
    fdc_id INTEGER PRIMARY KEY,       -- Unique permanent identifier of the food in the food table
    NDB_number INTEGER,                  -- Unique number assigned for the food (from SR Legacy)
    footnote TEXT,                    -- Comments on any unusual aspects; released to the public
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id),
    FOREIGN KEY (NDB_number) REFERENCES sr_legacy_food(NDB_number)
);
""")
conn.commit()
print("foundation_food table created.")




cur.execute("""
CREATE TABLE IF NOT EXISTS input_food (
    id INTEGER PRIMARY KEY,                    -- Unique permanent identifier for the input food record
    fdc_id INTEGER,                   -- fdc_id of the food that contains the input food
    fdc_id_of_input_food INTEGER,     -- fdc_id of the food that is the input food
    seq_num INTEGER,                           -- The order in which to display the input food
    amount REAL,                               -- The amount of the input food included within this food
    sr_code TEXT,                      -- The FF/SR (or FNDDS) code of the ingredient food
    sr_description TEXT,               -- The description of the ingredient food
    unit TEXT,                                 -- The unit of measure for the amount of the input food
    portion_code TEXT,                         -- Code identifying the portion description used to measure the ingredient
    portion_description TEXT,                  -- Description of the portion used to measure the ingredient
    gram_weight REAL,                          -- The weight in grams of the input food
    retention_code TEXT,                       -- Numeric code identifying processing on the input food
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id),
    FOREIGN KEY (fdc_id_of_input_food) REFERENCES food(fdc_id)
);
""")
conn.commit()
print("input_food table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS lab_method (
    id INTEGER PRIMARY KEY,         -- Unique permanent identifier for the lab method record
    description TEXT,               -- Description of the lab method
    technique TEXT                  -- General chemical analysis approach used by the lab method
);
""")
conn.commit()
print("lab_method table created.")




cur.execute("""
CREATE TABLE IF NOT EXISTS lab_method_code (
    lab_method_id INTEGER,         -- ID of the lab method the code refers to
    code TEXT,                     -- Value of the method code
    PRIMARY KEY (lab_method_id, code),
    FOREIGN KEY (lab_method_id) REFERENCES lab_method(id)
);
""")
conn.commit()
print("lab_method_code table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS lab_method_nutrient (
    lab_method_id INTEGER,             -- ID of the lab method the nutrient is measured by
    nutrient_id INTEGER,               -- ID of the nutrient that can be measured by the lab method
    PRIMARY KEY (lab_method_id, nutrient_id),
    FOREIGN KEY (lab_method_id) REFERENCES lab_method(id),
    FOREIGN KEY (nutrient_id) REFERENCES nutrient(id)
);
""")
conn.commit()
print("lab_method_nutrient table created.")




cur.execute("""
CREATE TABLE IF NOT EXISTS market_acquisition (
    id INTEGER PRIMARY KEY AUTOINCREMENT,   -- Unique permanent identifier for this acquisition record
    fdc_id INTEGER NOT NULL,                 -- ID of the food in the food table
    brand_description TEXT,                  -- Brand name description of the food
    expiration_date TEXT,                    -- Date the food will expire
    label_weight REAL,                       -- The weight of the food per the product label
    location TEXT,                           -- The region in which the food was purchased (e.g., "CA1")
    acquisition_date TEXT,                   -- Date the food was purchased
    sales_type TEXT,                         -- Type of establishment where the food was acquired (e.g., Retail Store, restaurant, farm)
    sample_lot_nbr TEXT,                     -- The lot number of the food
    sell_by_date TEXT,                       -- Date the food should be sold by
    store_city TEXT,                         -- City where the food was acquired
    store_name TEXT,                         -- Name of the store from which the food was purchased
    store_state TEXT,                        -- State where the food was acquired
    upc_code TEXT,                           -- UPC code for the food (only applicable for retail products)
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
);
""")
conn.commit()
print("market_acquisition table created.")



cur.execute("""
CREATE TABLE IF NOT EXISTS microbe (
    id INTEGER PRIMARY KEY AUTOINCREMENT,  -- Unique permanent identifier for the microbe record
    foodid INTEGER NOT NULL,                -- ID of the food; links to food(fdc_id)
    method TEXT,                          -- Method of analysis for the microbe count
    microbe_code TEXT,                    -- Code representing the scientific name of the microbe
    min_value REAL,                       -- Minimum value (colony forming units)
    max_value REAL,                       -- Maximum value (colony forming units)
    uom TEXT,                             -- Unit of measure for the microbe values
    FOREIGN KEY (foodid) REFERENCES food(fdc_id)
);
""")
conn.commit()
print("microbe table created.")



cur.execute("""
CREATE TABLE IF NOT EXISTS retention_factor (
    "n.gid" INTEGER PRIMARY KEY,             -- Unique permanent identifier for the retention factor record
    "n.code" TEXT,                -- Retention code as defined (e.g., from the USDA retention factor document)
    "n.foodGroupId" TEXT,               -- Code representing the food group for which this retention factor applies
    "n.description" TEXT          -- Description of the retention factor
);
""")
conn.commit()
print("retention_factor table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS sample_food (
    fdc_id INTEGER PRIMARY KEY,  -- Unique permanent identifier of the food in the food table for sample foods
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id)
);
""")
conn.commit()
print("sample_food table created.")




cur.execute("""
CREATE TABLE IF NOT EXISTS sub_sample_food (
    fdc_id INTEGER,                           -- ID of the food in the food table
    fdc_id_of_sample_food INTEGER,            -- ID of the sample food from which the sub sample originated
    PRIMARY KEY (fdc_id, fdc_id_of_sample_food),
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id),
    FOREIGN KEY (fdc_id_of_sample_food) REFERENCES food(fdc_id)
);
""")
conn.commit()
print("sub_sample_food table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS sub_sample_result (
    food_nutrient_id INTEGER PRIMARY KEY,  -- Unique ID for the row, same as the food_nutrient ID
    adjusted_amount REAL,                  -- Amount after adjusting for unit
    lab_method_id INTEGER,                 -- ID of the lab method used to measure the nutrient
    nutrient_name TEXT,                    -- The name of the nutrient as supplied by the lab
    FOREIGN KEY (food_nutrient_id) REFERENCES food_nutrient(id),
    FOREIGN KEY (lab_method_id) REFERENCES lab_method(id)
);
""")
conn.commit()
print("sub_sample_result table created.")




cur.execute("""
CREATE TABLE IF NOT EXISTS survey_fndds_food (
    fdc_id INTEGER PRIMARY KEY,         -- ID of the food in the food table
    food_code TEXT,                     -- Unique ID identifying the food within FNDDS
    wweia_category_code TEXT,           -- Unique identification number for the WWEIA food category
    start_date TEXT,                    -- Start date indicating time period corresponding to WWEIA data
    end_date TEXT,                      -- End date indicating time period corresponding to WWEIA data
    FOREIGN KEY (fdc_id) REFERENCES food(fdc_id), 
    FOREIGN KEY (wweia_category_code) REFERENCES wweia_food_category(wweia_food_category)
);
""")
conn.commit()
print("survey_fndds_food table created.")


cur.execute("""
CREATE TABLE IF NOT EXISTS wweia_food_category (
    wweia_food_category TEXT PRIMARY KEY,  -- Unique identification code for the WWEIA food category
    wweia_food_category_description TEXT
);
""")
conn.commit()
print("wweia_food_category table created.")

conn.close()

food table created.
measure_unit table created.
nutrient table created.
agricultural_samples table created.
acquisition_samples table created.
branded_food table created.
fndds_derivation table created.
fndds_ingredient_nutrient_value table created.
food_attribute table created.
food_attribute_type table created.
food_nutrient_conversion_factor table created.
food_calorie_conversion_factor table created.
food_category table created.
food_component table created.
food_nutrient table created.
food_nutrient_derivation table created.
food_nutrient_source table created.
food_portion table created.
food_protein_conversion_factor table created.
food_update_log_entry table created.
sr_legacy_food table created.
foundation_food table created.
input_food table created.
lab_method table created.
lab_method_code table created.
lab_method_nutrient table created.
market_acquisition table created.
microbe table created.
retention_factor table created.
sample_food table created.
sub_sample_food table 

# Importing CSV Data to the SQLite Database Tables

In [11]:
import sqlite3
import pandas as pd
import os
import logging

# Configure logging
logging.basicConfig(
    filename="import_errors.log",
    filemode="a",
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Path to your SQLite database file.
db_path = "FoodDataCentral.db"

# Subfolder containing the CSV files.
csv_folder = "FoodData_Central_csv_cleaned"

# List of table names that have corresponding CSV files.
table_names = [
    "food",
    "measure_unit",
    "nutrient",
    "agricultural_samples",
    "acquisition_samples",
    "branded_food",
    "fndds_derivation",
    "fndds_ingredient_nutrient_value",
    "food_attribute",
    "food_attribute_type",
    "food_nutrient_conversion_factor",
    "food_calorie_conversion_factor",
    "food_category",
    "food_component",
    "food_nutrient",
    "food_nutrient_derivation",
    "food_nutrient_source",
    "food_portion",
    "food_protein_conversion_factor",
    "food_update_log_entry",
    "sr_legacy_food",
    "foundation_food",
    "input_food",
    "lab_method",
    "lab_method_code",
    "lab_method_nutrient",
    "market_acquisition",
    "microbe",
    "retention_factor",
    "sample_food",
    "sub_sample_food",
    "sub_sample_result",
    "survey_fndds_food",
    "wweia_food_category"
]

# Connect to the SQLite database and enable foreign key support.
conn = sqlite3.connect(db_path)
conn.execute("PRAGMA foreign_keys = ON;")

for table in table_names:
    csv_file = os.path.join(csv_folder, f"{table}.csv")
    if os.path.exists(csv_file):
        print(f"Importing {csv_file} into {table} table...")
        try:
            # Read the CSV file with options to handle mixed types and bad lines.
            df = pd.read_csv(csv_file, low_memory=False, on_bad_lines='skip')
            
            # Import the DataFrame into the corresponding table.
            df.to_sql(table, conn, if_exists="append", index=False)
            
            # Commit after each successful table import.
            conn.commit()
            print(f"Successfully imported {csv_file} into {table}.")
        except Exception as e:
            error_message = f"Error importing {csv_file} into {table}: {e}"
            print(error_message)
            logging.error(error_message)
            conn.rollback()
    else:
        msg = f"CSV file for table '{table}' not found in {csv_folder}; skipping."
        print(msg)
        logging.error(msg)

conn.commit()
conn.close()
print("Data import complete.")


Importing FoodData_Central_csv_cleaned\food.csv into food table...
Successfully imported FoodData_Central_csv_cleaned\food.csv into food.
Importing FoodData_Central_csv_cleaned\measure_unit.csv into measure_unit table...
Successfully imported FoodData_Central_csv_cleaned\measure_unit.csv into measure_unit.
Importing FoodData_Central_csv_cleaned\nutrient.csv into nutrient table...
Successfully imported FoodData_Central_csv_cleaned\nutrient.csv into nutrient.
Importing FoodData_Central_csv_cleaned\agricultural_samples.csv into agricultural_samples table...
Successfully imported FoodData_Central_csv_cleaned\agricultural_samples.csv into agricultural_samples.
Importing FoodData_Central_csv_cleaned\acquisition_samples.csv into acquisition_samples table...
Successfully imported FoodData_Central_csv_cleaned\acquisition_samples.csv into acquisition_samples.
Importing FoodData_Central_csv_cleaned\branded_food.csv into branded_food table...
Successfully imported FoodData_Central_csv_cleaned\bran